In [7]:
# 导入DeepSeek API KEY
import os

os.environ["DEEPSEEK_API_KEY"]='<YOUR_DEEPSEEK_API_KEY>'
os.environ["LLM_MODEL"]='deepseek-chat'

In [5]:
pip install -i https://mirrors.aliyun.com/pypi/simple/ langchain_openai

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -i https://mirrors.aliyun.com/pypi/simple/ openai

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -i https://mirrors.aliyun.com/pypi/simple/ langchain

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install -i https://mirrors.aliyun.com/pypi/simple/ langchain-community

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# 提示词模版

In [36]:
from langchain import PromptTemplate, LLMChain
import os
from langchain_openai.chat_models.base import BaseChatOpenAI

def apply_prompt(sex, firstName):
    template_string = "我家是 {sex} 宝宝, 姓 {firstName}, 请帮忙起三个好听的名字。"
    prompt_template = PromptTemplate.from_template(template_string)
    prompt = prompt_template.format(sex=sex, firstName=firstName)

    llm = BaseChatOpenAI(
        model=os.getenv("LLM_MODEL"), 
        openai_api_key=os.getenv("DEEPSEEK_API_KEY"), 
        openai_api_base='https://api.deepseek.com',
        max_tokens=1024
    )

    response = llm.invoke(prompt)
    print(response.content)

In [37]:
apply_prompt(sex="女", firstName="闫")

给宝宝起名字是非常重要的事情，名字不仅要好听，还要有美好的寓意。以下是三个适合女宝宝的名字，供您参考：

1. **闫婉清**  
   - **寓意**：婉，温柔、婉约；清，清新、纯净。寓意宝宝温柔善良，清新脱俗。
   
2. **闫诗涵**  
   - **寓意**：诗，诗意、文雅；涵，涵养、包容。寓意宝宝有文采，气质优雅，心胸宽广。

3. **闫若曦**  
   - **寓意**：若，如同、仿佛；曦，晨曦、光明。寓意宝宝如晨曦般光彩照人，充满希望和光明。

希望这些名字能为您提供一些灵感，祝愿宝宝健康快乐成长！


# API调用链
## HTTP request chain

In [39]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chains import LLMRequestsChain
from langchain_openai.chat_models.base import BaseChatOpenAI

llm = BaseChatOpenAI(
        model=os.getenv("LLM_MODEL"), 
        openai_api_key=os.getenv("DEEPSEEK_API_KEY"), 
        openai_api_base='https://api.deepseek.com',
        max_tokens=1024
    )
def query_baidu(question):
      template = """Between >>> and <<< are the raw search result text from web.
      Extract the answer to the question '{query}' or say "not found" if the information is not contained.
      Use the format
      Extracted:<answer or "not found">
      >>> {requests_result} <<<
      Extracted:"""

      PROMPT = PromptTemplate(
          input_variables=["query", "requests_result"],
          template=template,
      )

      inputs = {
          "query": question,
          "url": "http://www.baidu.com/s?wd=" + question.replace(" ", "+")
      }
      requests_chain = LLMRequestsChain(llm_chain = LLMChain(llm=llm, prompt=PROMPT), output_key="query_info", verbose=True)
      res = requests_chain.run(inputs)
      return res

In [40]:
query_baidu("今天北京天气？")



> Entering new LLMRequestsChain chain...

> Finished chain.


'Extracted: 今天北京天气为晴，气温在-8~1°C之间，东北风2级，空气质量良。'

# 调用链

In [41]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import SequentialChain
from langchain.chat_models import AzureChatOpenAI
from langchain_openai.chat_models.base import BaseChatOpenAI

llm = BaseChatOpenAI(
        model=os.getenv("LLM_MODEL"), 
        openai_api_key=os.getenv("DEEPSEEK_API_KEY"), 
        openai_api_base='https://api.deepseek.com',
        max_tokens=1024
    )

summarizing_prompt_template = """
Summarize the following content into a sentence less than 20 words:
---
{content}

"""
summarizing_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(summarizing_prompt_template),
    output_key = "summary"
)

translating_prompt_template = """
translate "{summary}" into Chinese:

"""

translating_chain = LLMChain(
    llm = llm,
    prompt=PromptTemplate.from_template(translating_prompt_template),
    output_key = "translated"
)

overall_chain = SequentialChain(
    chains=[summarizing_chain, translating_chain],
    input_variables=["content"],
    output_variables=[ "summary","translated"],
    verbose=True
)


In [42]:
res = overall_chain("""
LangChain is a framework for developing applications powered by language models. It enables applications that are:

Data-aware: connect a language model to other sources of data
Agentic: allow a language model to interact with its environment
The main value props of LangChain are:

Components: abstractions for working with language models, along with a collection of implementations for each abstraction. Components are modular and easy-to-use, whether you are using the rest of the LangChain framework or not
Off-the-shelf chains: a structured assembly of components for accomplishing specific higher-level tasks
Off-the-shelf chains make it easy to get started. For more complex applications and nuanced use-cases, components make it easy to customize existing chains or build new ones.
""")

print("summary:"+res["summary"])

print("中文:"+res["translated"])

/var/folders/5v/lmyd52h516jglfw_vqn04sw80000gn/T/ipykernel_16975/2368733309.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = overall_chain("""




> Entering new SequentialChain chain...

> Finished chain.
summary:LangChain is a framework for building data-aware, agentic applications using modular components and pre-built chains for language models.
中文:"LangChain 是一个框架，用于构建数据感知、具有自主能力的应用程序，它利用模块化组件和预构建的语言模型链来实现这一目标。"
